# Differential Privacy
More information here:  https://github.com/tensorflow/privacy

## IMPORTANT:  You Should STOP All Kernels and Terminal Sessions
The GPU may be wedged at this point.  We stop all kernels and terminal sessions to release the GPU resources.

In [ ]:
!pip install numpy

# Calculate Privacy for Hyper-Parameters

Apply the RDP accountant to estimate privacy budget of an iterated Sampled Gaussian Mechanism. 

The output states that DP-SGD with these parameters satisfies (2.92, 1e-5)-DP.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math

from absl import app
from absl import flags

from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent

N = 600 # Total number of examples
batch_size = 32 # batch size
noise_multiplier = 1.12 # Noise multiplier for DP-SGD
epochs = 1 # Number of epochs (may be fractional)
delta = 1e-5 # Target delta

def apply_dp_sgd_analysis(q, sigma, steps, orders, delta):
  """Compute and print results of DP-SGD analysis."""

  rdp = compute_rdp(q, sigma, steps, orders)

  eps, _, opt_order = get_privacy_spent(orders, rdp, target_delta=delta)

  print('DP-SGD with sampling rate = {:.3g}% and noise_multiplier = {} iterated'
        ' over {} steps satisfies'.format(100 * q, sigma, steps), end=' ')
  print('differential privacy with eps = {:.3g} and delta = {}.'.format(
      eps, delta))
  print('The optimal RDP order is {}.'.format(opt_order))

  if opt_order == max(orders) or opt_order == min(orders):
    print('The privacy estimate is likely to be improved by expanding '
          'the set of orders.')

q = batch_size / N  # q - the sampling ratio.

if q > 1:
  raise app.UsageError('N must be larger than the batch size.')

orders = ([1.25, 1.5, 1.75, 2., 2.25, 2.5, 3., 3.5, 4., 4.5] + list(range(5, 64)) +
            [128, 256, 512])

steps = int(math.ceil(epochs * N / batch_size))

apply_dp_sgd_analysis(q, noise_multiplier, steps, orders, delta)


# Train with Differential Privacy SGD

In [ ]:
"""Training a CNN on MNIST with differentially private SGD optimizer."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers import dp_optimizer

dpsgd = True # If True, train with DP-SGD. If False, train with vanilla SGD.
learning_rate = 0.08 # Learning rate for training
noise_multiplier = 1.12 # Ratio of the standard deviation to the clipping norm
l2_norm_clip = 1.0 # Clipping norm
batch_size = 32 # Batch size
epochs = 1 # Number of epochs
microbatches = 32 # Number of microbatches (must evenly divide batch_size
model_dir = None # Model directory
export_dir = './pipeline_tfserving/0' # Export dir

In [ ]:
import numpy as np
import tensorflow as tf

def cnn_model_fn(features, labels, mode):
  """Model function for a CNN."""

  # Define CNN architecture using tf.keras.layers.
  input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
  y = tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             kernel_initializer='he_normal').apply(input_layer)
  y = tf.keras.layers.MaxPool2D(2, 1).apply(y)
  y = tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             kernel_initializer='he_normal').apply(y)
  y = tf.keras.layers.MaxPool2D(2, 1).apply(y)
  y = tf.keras.layers.Flatten().apply(y)
  y = tf.keras.layers.Dense(32, kernel_initializer='he_normal').apply(y)
  logits = tf.keras.layers.Dense(10, kernel_initializer='he_normal').apply(y)

  # Calculate loss as a vector (to support microbatches in DP-SGD).
  vector_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits)
  # Define mean of loss across minibatch (for reporting through tf.Estimator).
  scalar_loss = tf.reduce_mean(vector_loss)

  # Configure the training op (for TRAIN mode).
  if mode == tf.estimator.ModeKeys.TRAIN:

    if dpsgd:
      # Use DP version of GradientDescentOptimizer. For illustration purposes,
      # we do that here by calling make_optimizer_class() explicitly, though DP
      # versions of standard optimizers are available in dp_optimizer.
      dp_optimizer_class = dp_optimizer.make_optimizer_class(
          tf.train.GradientDescentOptimizer)
      optimizer = dp_optimizer_class(
          learning_rate=learning_rate,
          noise_multiplier=noise_multiplier,
          l2_norm_clip=l2_norm_clip,
          num_microbatches=microbatches)
      opt_loss = vector_loss
    else:
      optimizer = tf.train.GradientDescentOptimizer(
          learning_rate=learning_rate)
      opt_loss = scalar_loss
    global_step = tf.train.get_global_step()
    train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)
    # In the following, we pass the mean of the loss (scalar_loss) rather than
    # the vector_loss because tf.estimator requires a scalar loss. This is only
    # used for evaluation and debugging by tf.estimator. The actual loss being
    # minimized is opt_loss defined above and passed to optimizer.minimize().
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=scalar_loss,
                                      train_op=train_op)

  # Add evaluation metrics (for EVAL mode).
  elif mode == tf.estimator.ModeKeys.EVAL:
    eval_metric_ops = {
        'accuracy':
            tf.metrics.accuracy(
                labels=labels,
                predictions=tf.argmax(input=logits, axis=1))
    }
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=scalar_loss,
                                      eval_metric_ops=eval_metric_ops)


def load_mnist():
  """Loads MNIST and preprocesses to combine training and validation data."""
  train, test = tf.keras.datasets.mnist.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.
  assert len(train_labels.shape) == 1
  assert len(test_labels.shape) == 1

  return train_data, train_labels, test_data, test_labels



tf.logging.set_verbosity(tf.logging.INFO)
if batch_size % microbatches != 0:
  raise ValueError('Number of microbatches should divide evenly batch_size')

# Load training and test data.
train_data, train_labels, test_data, test_labels = load_mnist()

# Instantiate the tf.Estimator.
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                            model_dir=model_dir)

# Create tf.Estimator input functions for the training and test data.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={'x': train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=epochs,
      shuffle=True)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={'x': test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)

# Define a function that computes privacy budget expended so far.
def compute_epsilon(steps):
  """Computes epsilon value for given hyperparameters."""
  if noise_multiplier == 0.0:
    return float('inf')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = batch_size / N
  rdp = compute_rdp(q=sampling_probability,
                      noise_multiplier=noise_multiplier,
                      steps=steps,
                      orders=orders)
    # Delta is set to 1e-5 because MNIST has N training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

# Training loop.
steps_per_epoch = N // batch_size
for epoch in range(1, epochs + 1):
  # Train the model for one epoch.
  mnist_classifier.train(input_fn=train_input_fn, steps=steps_per_epoch)

  # Evaluate the model and print results
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  test_accuracy = eval_results['accuracy']
  print('Test accuracy after %d epochs is: %.3f' % (epoch, test_accuracy))

  # Compute the privacy budget expended so far.
  if dpsgd:
    eps = compute_epsilon(epoch * steps_per_epoch)
    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
  else:
    print('Trained with vanilla non-private SGD optimizer')